In [59]:
#라이브러리 생성
import tensorflow as tf
import pandas as pd


In [60]:
#과거 데이터 준비
파일경로 = 'https://raw.githubusercontent.com/blackdew/tensorflow1/master/csv/boston.csv'
보스턴 = pd.read_csv(파일경로)
print(보스턴.columns)
보스턴.head()


Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'b', 'lstat', 'medv'],
      dtype='object')


,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [61]:
#종속변수, 독립변수 나누기

종속 = 보스턴[['medv']]
독립 = 보스턴[['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'b', 'lstat']]
print(종속.shape,독립.shape)

(506, 1) (506, 13)


In [62]:
#모델 만들기

x = tf.keras.layers.Input(shape=[13])
#hidden layer 를 추가한다. 
h = tf.keras.layers.Dense(10,activation='swish')(x)
#마지막 output값에 입력되는 것은 hidden layer 이다.
y = tf.keras.layers.Dense(1)(h)

model = tf.keras.models.Model(x,y)
model.compile(loss='mse')


In [63]:
#batchnormaization 을 이용해 모델을 만들어준다
#성능이 훨씬 좋아진다.


x = tf.keras.layers.Input(shape=[13])

#hidden layer 를 추가한다. 
#batchnormalization 을 이용하여 모델을 구성
h = tf.keras.layers.Dense(10)(x)
h = tf.keras.layers.BatchNormalization()(h)
h = tf.keras.layers.Activation('swish')(h)

h = tf.keras.layers.Dense(10)(h)
h = tf.keras.layers.BatchNormalization()(h)
h = tf.keras.layers.Activation('swish')(h)

h = tf.keras.layers.Dense(10)(h)
h = tf.keras.layers.BatchNormalization()(h)
h = tf.keras.layers.Activation('swish')(h)

#마지막 output값에 입력되는 것은 hidden layer 이다.
y = tf.keras.layers.Dense(1)(h)

model = tf.keras.models.Model(x,y)
model.compile(loss='mse')


In [64]:
#모델이 원하는 구조로로 만들어졌는지 확인하기
#13개의 input과 하나의 bias가 hidden 10개로 만들어진다.
#(13+1) * 10 = 140

model.summary()

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 13)]              0         
_________________________________________________________________
dense_26 (Dense)             (None, 10)                140       
_________________________________________________________________
batch_normalization_6 (Batch (None, 10)                40        
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                110       
_________________________________________________________________
batch_normalization_7 (Batch (None, 10)                40        
_________________________________________________________________
activation_7 (Activation)    (None, 10)              

In [65]:
#모델을 학습시킨다.

model.fit(독립,종속,epochs=100,verbose=0)
model.fit(독립,종속,epochs=10)



Epoch 1/10
16/16 [==============================] - 0s 2ms/step - loss: 28.1351
Epoch 2/10
16/16 [==============================] - 0s 1ms/step - loss: 21.5744
Epoch 3/10
16/16 [==============================] - 0s 1ms/step - loss: 22.7305
Epoch 4/10
16/16 [==============================] - 0s 1ms/step - loss: 19.0679
Epoch 5/10
16/16 [==============================] - 0s 1ms/step - loss: 22.7089
Epoch 6/10
16/16 [==============================] - 0s 1ms/step - loss: 21.1398
Epoch 7/10
16/16 [==============================] - 0s 1ms/step - loss: 24.7760
Epoch 8/10
16/16 [==============================] - 0s 1ms/step - loss: 22.8375
Epoch 9/10
16/16 [==============================] - 0s 2ms/step - loss: 20.7761
Epoch 10/10
16/16 [==============================] - 0s 1ms/step - loss: 20.7042


In [66]:
#모델을 이용해 독립값(원인)으로 종속(결과)값 구하기

print(model.predict(독립[:5]))#0부터5까지 
print(종속[:5])


[[26.716467]
 [20.646404]
 [26.510946]
 [29.014395]
 [25.84576 ]]
   medv
0  24.0
1  21.6
2  34.7
3  33.4
4  36.2


In [67]:
# 모델의 수식 확인
print(model.get_weights())

[array([[-0.26379505,  0.28386906,  0.04725728,  0.11799081,  0.2368915 ,
         0.07235741, -0.20175461,  0.04061565,  0.22403723,  0.2643932 ],
       [ 0.31740594,  0.10884423,  0.02753879,  0.14652282,  0.00614315,
        -0.09761841,  0.5268601 , -0.41744214, -0.21009919,  0.5106822 ],
       [ 0.05357186,  0.70079535, -0.48226857,  0.16144586,  0.47423536,
        -0.100793  , -0.45933628,  0.08639682, -0.20859773,  0.08005802],
       [ 0.22421634, -0.03432721,  0.12767017, -0.9642347 ,  0.23059046,
         0.7506756 , -0.00290061, -0.8719852 , -0.36160484, -0.00702212],
       [ 0.07383252, -0.03678077, -0.99063337, -0.05497634,  0.37180114,
        -0.15010609,  0.44148552,  0.6056538 ,  0.5582165 , -0.01559841],
       [ 0.5704537 ,  0.57915294, -0.6274952 , -1.210094  ,  0.60325956,
         0.93481976,  1.2548832 , -0.91796386, -0.49889562, -0.960766  ],
       [ 0.49420825,  0.30021122,  0.3103409 ,  0.2685867 ,  0.5850859 ,
        -0.396712  ,  0.12870012,  0.2869402

In [68]:
#아이리스 품종 분류 -> 종속이 범주(이름)로 되어있어 원핫인코딩 필요

파일경로 = 'https://raw.githubusercontent.com/blackdew/tensorflow1/master/csv/iris.csv'
아이리스 = pd.read_csv(파일경로)


In [69]:
#원핫인코딩을 이용해 종속값에 있는 이름을 숫자형으로 변환

아이리스 = pd.get_dummies(아이리스)
print(아이리스.columns)
아이리스.head()


Index(['꽃잎길이', '꽃잎폭', '꽃받침길이', '꽃받침폭', '품종_setosa', '품종_versicolor',
       '품종_virginica'],
      dtype='object')


,꽃잎길이,꽃잎폭,꽃받침길이,꽃받침폭,품종_setosa,품종_versicolor,품종_virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


In [70]:
#종속변수, 독립변수 나눠주가

독립 = 아이리스[['꽃잎길이', '꽃잎폭', '꽃받침길이', '꽃받침폭']]
종속 = 아이리스[['품종_setosa', '품종_versicolor','품종_virginica']]

print(독립.shape,종속.shape)

(150, 4) (150, 3)


In [71]:
#hidden layer를 포함한 모델 구조 설계
#output에 꼭!! softmax적용시켜줘야함.

x = tf.keras.layers.Input(shape = 4)
h = tf.keras.layers.Dense(8,activation='swish')(x)
h = tf.keras.layers.Dense(8,activation='swish')(h)
h = tf.keras.layers.Dense(8,activation='swish')(h)
y = tf.keras.layers.Dense(3,activation='softmax')(h)

model = tf.keras.models.Model(x,y)
#종속변수가 범주형이기 때문에 loss를 msc가 아닌 entropy 써주고, 정확도를 보기위해 accuracy를 설정해준다.
model.compile(loss='categorical_crossentropy',metrics='accuracy')


In [72]:
#hidden layer를 포함한 모델 구조 설계
#output에 꼭!! softmax적용시켜줘야함.

x = tf.keras.layers.Input(shape = 4)

h = tf.keras.layers.Dense(8)(x)
h = tf.keras.layers.BatchNormalization()(h)
h = tf.keras.layers.Activation('swish')(h)

h = tf.keras.layers.Dense(8)(h)
h = tf.keras.layers.BatchNormalization()(h)
h = tf.keras.layers.Activation('swish')(h)

h = tf.keras.layers.Dense(8)(h)
h = tf.keras.layers.BatchNormalization()(h)
h = tf.keras.layers.Activation('swish')(h)


y = tf.keras.layers.Dense(3,activation='softmax')(h)

model = tf.keras.models.Model(x,y)
#종속변수가 범주형이기 때문에 loss를 msc가 아닌 entropy 써주고, 정확도를 보기위해 accuracy를 설정해준다.
model.compile(loss='categorical_crossentropy',metrics='accuracy')


In [73]:
model.summary()

Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
dense_34 (Dense)             (None, 8)                 40        
_________________________________________________________________
batch_normalization_9 (Batch (None, 8)                 32        
_________________________________________________________________
activation_9 (Activation)    (None, 8)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 8)                 72        
_________________________________________________________________
batch_normalization_10 (Batc (None, 8)                 32        
_________________________________________________________________
activation_10 (Activation)   (None, 8)               

In [74]:
#모델을 학습시킨다.

model.fit(독립,종속,epochs=1000,verbose = 0)
model.fit(독립,종속,epochs=10)


Epoch 1/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0535 - accuracy: 0.9733
Epoch 2/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0345 - accuracy: 0.9800
Epoch 3/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0537 - accuracy: 0.9800
Epoch 4/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0322 - accuracy: 0.9867
Epoch 5/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0388 - accuracy: 0.9867
Epoch 6/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0287 - accuracy: 0.9933
Epoch 7/10
5/5 [==============================] - 0s 2ms/step - loss: 0.1095 - accuracy: 0.9533
Epoch 8/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0660 - accuracy: 0.9667
Epoch 9/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0288 - accuracy: 0.9933
Epoch 10/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0785 - accuracy: 0.9733


In [75]:
#모델을 이용해 학습잘 되었나 확인하기

print(종속[:5])#나와야 하는 결과
model.predict(독립[:5])


   품종_setosa  품종_versicolor  품종_virginica
0          1              0             0
1          1              0             0
2          1              0             0
3          1              0             0
4          1              0             0


array([[9.9998689e-01, 1.2708630e-05, 3.3395079e-07],
       [9.9964309e-01, 3.5655900e-04, 3.5542450e-07],
       [9.9997103e-01, 2.8601436e-05, 3.2873021e-07],
       [9.9990559e-01, 9.4087169e-05, 4.0902529e-07],
       [9.9999690e-01, 2.4966241e-06, 6.2382696e-07]], dtype=float32)